## Big Mart Sales Prediction

### Research Question

Through the analysis, I am planning to predict the impact of other factors on sales of a particular product in a particular store. The train file will be used to explore the data and train the model. The model will be applied on test data in the test file and model performance will be evaluated.

### Data Source

The data has been collected from [here](https://code.datasciencedojo.com/tshrivas/dojoHub/tree/a152a17dee24dcfcc10bb75c77c2e88cdcf90212/Big%20Mart%20Sales%20DataSet)

In [5]:
# import libraries

import pandas as pd
import numpy as np
import pandas_profiling
import warnings

from scipy.stats import mode

# suppress unnecessary warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [9]:
# data location

dirData = 'Data/'
fileTrain = 'train.csv'
fileTest = 'test.csv'

# read files

train = pd.read_csv(dirData+fileTrain)
test = pd.read_csv(dirData+fileTest)

Since the source files are already divided into train and test, following machine learning best practices, I decided to merge them, yet keeping an indicator. This way, feature engineering could be applied on the entire dataset prior to training and testing.

In [66]:
# add identifier columns for futureuse
train['source']='train'
test['source']='test'

# merge data
sales_df = pd.concat([train, test],ignore_index=True, sort=False)

# check row count
print('Train: ',train.shape)
print('Test: ',test.shape)
print('Sales: ',sales_df.shape)

# check merged data
sales_df.head()

Train:  (8523, 13)
Test:  (5681, 12)
Sales:  (14204, 13)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


### Exploratory Analysis

In [11]:
# create data profile
pandas_profiling.ProfileReport(sales_df)

Number of variables,13
Number of observations,14204
Total Missing (%),6.6%
Total size in memory,1.4 MiB
Average record size in memory,104.0 B
Numeric,5
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0


Analysis of each variable in the dataset might be redundant. Therefore, documenting the interesting observations as below:
    
* `Item_Fat_Content`: The variable has _6.6%_ missing values. But what is more important is it needs some data cleansing. The unique values contain 'Low Fat', 'LF', 'low fat' and 'Regular', 'reg'. They need to be encoded correctly in the data cleansing process.

* `Item_Identifier`: There are 1559 distinct values, meaning we have data for 10 different items.

* `Outlet_Identifier`: There are 10 distinct values, meaning we have data for 10 different stores.

* `Item_MRP`: The extreme values range from 31.29 to 266.89. This is a critical component in sales prediction. It is good that there is no missing value.

* `Item_Type`: Item type has 16 unique values, i.e. there are 16 different item types in the data.

* `Item_Visibility`: I was expecting this to be categorical, but visibility is denoted by numeric values. I may or may not change it in future based on subsequent findings.

* `Item_Outlet_Sales`: It is interesting that this variable is missing _40%_ of the values. But a closer look will tell that this is the target variable that we are trying to predict in the test dataset. So the missing values are perfectly ok.

* `Outlet_Size`: Interestingly, this categorical variable is missing _28.3%_ values.

### Data Cleansing

#### Step 1:

Since `Item_Weight` has _17.2%_ missing values, I decided to populate those values with average item weight to have a cleaner dataset without losing data. From the profie, average weight of an item is __12.793__ and this is the value I will use to populate the blanks.

In [58]:
print('Missing Item_Weight before cleansing: ', sum(sales_df['Item_Weight'].isnull()))

# calculate mean weight to fill in the blanks and fill NAs
mean_weight = sales_df['Item_Weight'].mean()

sales_df[['Item_Weight']] = sales_df[['Item_Weight']].fillna(value=mean_weight)

print('Missing Item_Weight after cleansing: ', sum(sales_df['Item_Weight'].isnull()))

Missing Item_Weight before cleansing:  2439
Missing Item_Weight after cleansing:  0


#### Step 2:

Since `Outlet_Size` has _28.3%_ missing values, they need to be taken care of. But unlike `Item_Weight`, `Outlet_Size` is a categorical variable, hence I cannot use mean to fill up. Instead, I am using the most frequent value, i.e. mode to fill up the blanks.

In [59]:
print('Missing Outlet_Size before cleansing: ', sum(sales_df['Outlet_Size'].isnull()))

# calculate mean weight to fill in the blanks and fill NAs
mode_size = str(sales_df['Outlet_Size'].mode()[0])

sales_df[['Outlet_Size']] = sales_df[['Outlet_Size']].fillna(value=mode_size)

print('Missing Outlet_Size after cleansing: ', sum(sales_df['Outlet_Size'].isnull()))

Missing Outlet_Size before cleansing:  4016
Missing Outlet_Size after cleansing:  0


#### Step 3: 

Since `Item_Visibility` has _6.2%_ zero values, they need to be cleaned up as well. I will trerat this same as `Item_Weight` and use mean weight to fill up.

In [67]:
print('Missing Item_Visibility before cleansing: ', sum(sales_df['Item_Visibility'] == 0))

# calculate mean visibility to fill in the blanks and fill 0s
mean_visibility = sales_df['Item_Visibility'].mean()

conditions = [
    sales_df['Item_Visibility'] == 0,
    sales_df['Item_Visibility'] != 0
]

choices = [mean_visibility, sales_df['Item_Visibility']]

sales_df['Item_Visibility'] = np.select(conditions, choices, default = 0)

print('Missing Item_Visibility after cleansing: ', sum(sales_df['Item_Visibility'] == 0))

Missing Item_Visibility before cleansing:  879
Missing Item_Visibility after cleansing:  0


### Feature Engineering

Now that we have a cleaner dataset, we would concentrate on modifying and/or adding features that are beneficial for model formulation.

#### Step 1

As we can see, there are _16_ different item types, but many of them belong to the same category, e.g. _Snack Foods_, _Frozen Foods_ etc. could be combined as foods. We can use `Item_Identifier` for a more logical approach. First two characters of `Item_Identifier` has the following distinct values:

In [80]:
# get first 2 characters of the Item_Identifier to frame the categories

sales_df['Item_Identifier'].astype(str).str[:2].unique()

array(['FD', 'DR', 'NC'], dtype=object)

To create a broad category, I would encode 'FD' as 'Food', 'DR' as 'Drink' and 'NC' as 'Non-Consumable'.

In [78]:
# making sure the counts match

sales_df['Item_Identifier'].astype(str).str[:2].value_counts()

FD    10201
NC     2686
DR     1317
Name: Item_Identifier, dtype: int64

In [81]:
conditions = [
    (sales_df['Item_Identifier'].astype(str).str[:2] == 'FD'),
    (sales_df['Item_Identifier'].astype(str).str[:2] == 'DR'),
    (sales_df['Item_Identifier'].astype(str).str[:2] == 'NC')
]

choices = ['Food', 'Drink', 'Non-Consumable']

sales_df['Item_Category'] = np.select(conditions, choices, default='')

# making sure the counts match

sales_df['Item_Category'].value_counts()

Food              10201
Non-Consumable     2686
Drink              1317
Name: Item_Category, dtype: int64

#### Step 2: 
Standerdize the value of the categorical variable, `Item_Fat_Content`

In [82]:
# check different values of Item Fat Content
sales_df['Item_Fat_Content'].unique()

conditions = [
    (sales_df.Item_Fat_Content == 'LF') | (sales_df.Item_Fat_Content.str.upper() == 'LOW FAT'),
    (sales_df.Item_Fat_Content.str.upper() == 'REG') | (sales_df.Item_Fat_Content.str.upper() == 'REGULAR')
]

choices = ['Low Fat', 'Regular']

sales_df['Item_Fat_Content'] = np.select(conditions, choices, default='')

# check update
sales_df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)